#**6th Week**

#**Задачи (Оба уровня)**

Тест состоит из задач по теме "Классы. Магические методы". Вам предстоит решить ряд задач, демонстрирующих ваше понимание пройденных тем.  Вы можете проходить тест неограниченное количество раз, и в зачет идет ваш лучший результат. Удачи!



##**Эмуляция контейнеров**

Напишем свой аналог листа таблицы Excel. Нужно написать структуру данных Field, в которой доступ к значениям будет осуществляться по ключам. Ключом будет пара "буква" - "число", по аналогии с адресом ячейки в Excel. Возможные форматы обращения к одной и той же "ячейке" данных:

```
field = Field()

field[1, 'a'] = 25
field['a', 1] = 25
field['a', '1'] = 25
field['1', 'a'] = 25
field['1a'] = 25
field['a1'] = 25
field[1, 'A'] = 25 # Все то же самое работает и с заглавными буквами
field['A', 1] = 25
field['A', '1'] = 25
field['1', 'A'] = 25
field['1A'] = 25
field['A1'] = 25
```

В этом списке каждая из этих строк записывает число 25 в ячейку с одним и тем же ключом. Соответственно, по любому из перечисленных ключей должно быть можно получить это число из объекта field. Также должны быть реализованы удаление элемента из структуры (через оператор del) и возможность использования оператора in, например:

```
(1, 'a') in field: True
"A1" in field: True
('D', '4') in field: False
```

Таким образом, выходит, что ключом структуры может быть либо кортеж, либо строка. При попытке получить или записать значение по ключу другого типа должно быть вызвано исключение TypeError. При некорректном значении строки или элементов кортежа нужно вызывать исключение ValueError. Корректными значениями будет считать одиночные буквы и неотрицательное целое число любой длины, т.е. правильные варианты ключей:

- А1
- А222543
- Z89

Неправильные варианты ключей:

- AA5
- Q2.5
- -6F
- A
- 27
- GG

Кроме вышеперчисленного, по объекту должно быть возможно итерироваться. При проходе циклом по объекту должны возвращаться значения, хранящиеся в нём. Порядок возврата значений не важен.

Если запрашивается правильный формат ячейки, но в нашем контейнере такого ключа нет, то нужно вернуть None. Например:
```
field = Field()
print(field["C5"] is None)  # Выводит: True
```

###**Примечания**

В своем решении этого задания я использовал в качестве ключей хранимого словаря frozenset, а проверку на ValueError реализовал через регулярку. Также рекомендую проверку типов и преобразование поступившего ключа в тот вид, в котором он хранится "под капотом", вынести в отдельный метод и вызывать его из всех описываемых магических методов.



In [ ]:
import re

class Field:
    def __init__(self):
        self._data = {}

    @staticmethod
    def _normalize_key(key):
        if isinstance(key, tuple):
            if len(key) != 2:
                raise ValueError("Key tuple must have exactly two elements.")
            part1, part2 = key
            # Определяем букву и число независимо от их порядка
            if isinstance(part1, str) and part1.isdigit() and isinstance(part2, str) and part2.isalpha():
                number, letter = part1, part2
            elif isinstance(part2, str) and part2.isdigit() and isinstance(part1, str) and part1.isalpha():
                letter, number = part1, part2
            elif isinstance(part1, int) and isinstance(part2, str) and part2.isalpha():
                number, letter = str(part1), part2
            elif isinstance(part2, int) and isinstance(part1, str) and part1.isalpha():
                letter, number = part1, str(part2)
            else:
                raise ValueError("Invalid tuple key format.")
        elif isinstance(key, str):
            # Проверяем форматы "буква + число" или "число + буква"
            match = re.match(r"^([a-zA-Z])(\d+)$", key)
            if match:
                letter, number = match.groups()
            else:
                match = re.match(r"^(\d+)([a-zA-Z])$", key)
                if not match:
                    raise ValueError("Invalid string key format.")
                number, letter = match.groups()
        else:
            raise TypeError("Key must be a tuple or a string.")

        # Приводим букву к верхнему регистру и число к целому типу
        letter = letter.upper()
        if len(letter) != 1 or not letter.isalpha():
            raise ValueError("Invalid key format: letter must be a single alphabetic character.")
        if not str(number).isdigit():
            raise ValueError("Invalid key format: number must be numeric.")
        number = int(number)

        return (letter, number)

    def __setitem__(self, key, value):
        normalized_key = self._normalize_key(key)
        self._data[normalized_key] = value

    def __getitem__(self, key):
        normalized_key = self._normalize_key(key)
        return self._data.get(normalized_key, None)

    def __delitem__(self, key):
        normalized_key = self._normalize_key(key)
        if normalized_key in self._data:
            del self._data[normalized_key]

    def __contains__(self, key):
        normalized_key = self._normalize_key(key)
        return normalized_key in self._data

    def __iter__(self):
        return iter(self._data.values())


##**Доступ к атрибутам**

Доработать класс Field так, чтобы вдобавок к реализованному функционалу появились следующие возможности:
```
field = Field()

Запись значения в ячейку:
field.a1 = 25: эквивалентно field['a1'] = 25
field.A1 = 25: то же самое
```
Получение значения:
```
field['b', 2] = 100
field.b2
field.B2
```
Удаление значения:
```
del field.a1, del field.A1 - эквивалентно del field['a', 1]
```

Таким образом, внутри класса Field методы работы с атрибутами должны работать с тем же объектом, в котором хранятся значения, обрабатываемые в методах `__setitem__, __getitem__, __delitem__`.

Кроме того, обычное присвоение и получение атрибутов (тех, которые не являются адресом ячейки данных нашего класса) должно производиться по стандартному алгоритму питоновских объектов, т.е. они должны храниться в словаре `__dict__` объекта.
```
field = Field()
field.abcde = 125
print(field.abcde, field.__dict__['abcde'] == 125)  # Выводит: 125 True
```

Для таких атрибутов также должны быть реализованы получение, присваивание и удаление значения.



In [ ]:
import re

class Field:
    def __init__(self):
        self._data = {}

    @staticmethod
    def _normalize_key(key):
        if isinstance(key, tuple):
            if len(key) != 2:
                raise ValueError("Key tuple must have exactly two elements.")
            part1, part2 = key
            # Определяем букву и число независимо от их порядка
            if isinstance(part1, str) and part1.isdigit() and isinstance(part2, str) and part2.isalpha():
                number, letter = part1, part2
            elif isinstance(part2, str) and part2.isdigit() and isinstance(part1, str) and part1.isalpha():
                letter, number = part1, part2
            elif isinstance(part1, int) and isinstance(part2, str) and part2.isalpha():
                number, letter = str(part1), part2
            elif isinstance(part2, int) and isinstance(part1, str) and part1.isalpha():
                letter, number = part1, str(part2)
            else:
                raise ValueError("Invalid tuple key format.")
        elif isinstance(key, str):
            # Проверяем форматы "буква + число" или "число + буква"
            match = re.match(r"^([a-zA-Z])(\d+)$", key)
            if match:
                letter, number = match.groups()
            else:
                match = re.match(r"^(\d+)([a-zA-Z])$", key)
                if not match:
                    raise ValueError("Invalid string key format.")
                number, letter = match.groups()
        else:
            raise TypeError("Key must be a tuple or a string.")

        # Приводим букву к верхнему регистру и число к целому типу
        letter = letter.upper()
        if len(letter) != 1 or not letter.isalpha():
            raise ValueError("Invalid key format: letter must be a single alphabetic character.")
        if not str(number).isdigit():
            raise ValueError("Invalid key format: number must be numeric.")
        number = int(number)

        return (letter, number)

    def __setitem__(self, key, value):
        normalized_key = self._normalize_key(key)
        self._data[normalized_key] = value

    def __getitem__(self, key):
        normalized_key = self._normalize_key(key)
        return self._data.get(normalized_key, None)

    def __delitem__(self, key):
        normalized_key = self._normalize_key(key)
        if normalized_key in self._data:
            del self._data[normalized_key]

    def __contains__(self, key):
        normalized_key = self._normalize_key(key)
        return normalized_key in self._data

    def __iter__(self):
        return iter(self._data.values())

    def __setattr__(self, name, value):
        # Проверка, если имя атрибута является ключом ячейки
        if re.match(r"^[a-zA-Z]\d+$", name):  # Формат 'a1', 'A2' и т.д.
            self.__dict__[name] = value  # Мы используем стандартный __dict__ для атрибутов
            self[name] = value  # Приводим это к соответствующему методу __setitem__
        else:
            super().__setattr__(name, value)  # Для обычных атрибутов применяем стандартный метод

    def __getattr__(self, name):
        # Если атрибут является ключом ячейки
        if re.match(r"^[a-zA-Z]\d+$", name):  # Формат 'a1', 'A2' и т.д.
            return self[name]  # Используем метод __getitem__ для получения значения
        # Для обычных атрибутов будет использоваться стандартное поведение
        raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")

    def __delattr__(self, name):
        # Если атрибут является ключом ячейки
        if re.match(r"^[a-zA-Z]\d+$", name):  # Формат 'a1', 'A2' и т.д.
            del self[name]  # Приводим это к методу __delitem__
        else:
            super().__delattr__(name)  # Для обычных атрибутов применяем стандартный метод


##**Исключения**

Напишем часть сервиса, который будет помогать бронировать переговорки в офисе. Для этого опишем класс Booking - его объекты будут содержать информацию о конкретном бронировании, а также вспомогательную функцию create_booking, которая будет создавать новый объект бронирования и записывать информацию о бронировании в базу данных бронирований через предоставляемое API. Возвращать она должна будет статус создания бронирования (получилось или переговорка уже занята) и информацию о брони в формате JSON. Ниже - подробности.

Класс `Booking` должен обладать следующим функционалом.

- конструктор должен принимать три аргумента в следующем порядке: название переговорки, datetime начала брони и datetime конца брони

- внутри конструктора, если datetime конца брони оказался раньше, чем datetime начала, нужно вызвать исключение ValueError

Также у объектов этого класса должны быть следующие поля (рекомендую сделать часть из них в виде проперти):

- room_name - название переговорки, полученное из конструктора

- start - datetime начала брони. Должна быть возможность назначить новое время начала уже созданной брони

- end - datetime конца брони. Должна быть возможность назначить новое время конца уже созданной брони

- duration - длительность бронирования в минутах (гарантируется, что длительность любой встречи кратна одной минуте, поэтому это должно быть целое число)

- start_date - дата начала брони в формате YYYY-MM-DD (строка)

- end_date - дата конца брони в формате YYYY-MM-DD (строка)

- start_time - время начала брони в формате HH:MM (строка)

- end_time - время конца брони в формате HH:MM (строка)

Функция create_booking должна обладать следующей сигнатурой:
```
create_booking(room_name, start, end) -> str,
```
где аргументы - это те же аргументы, которые принимает конструктор Booking, а выходная строка - это json определенного формата, который описан чуть ниже по тексту.

Будем считать, что взаимодействие с базой данных у нас уже описано нашим коллегой в соседнем файле api.py. В нем есть уже готовая функция `register_booking`, которая:

- принимает на вход объект класса `Booking`

- возвращает True, если бронирование получилось создать

- возвращает False, если мы пытаемся забронировать уже занятую в это время переговорку

- если такой переговорки не существует, вызывается KeyError

Таким образом, в том же файле, что и класс Booking, вам нужно описать фукнцию create_booking, которая:

- обладает сигнатурой create_booking(room_name, start, end) -> str, где аргументы - те же, что и в конструкторе Booking

- в самом начале своей работы выводит на экран текст: Начинаем создание бронирования

- внутри функции создается объект класса Booking, а также вызывается функция register_booking, которая принимает на вход созданный объект. Должны быть обработаны все случаи работы register_booking: True, False и KeyError. Сделать это поможет конструкция try-except

- перед выходом из функции должно выводиться на экран сообщение Заканчиваем создание бронирования. Это должно происходить в любом случае, даже если мы попытались создать бронирование с неверными датами и получили ValueError (см. описание класса Booking). Для этого рекомендую использовать блок finally, в котором описать этот print

Функция должна возвращать json-строку с ответом, в котором будут содержаться следующие поля:

- created: true/false, получилось ли забронировать комнату. Если возникло KeyError, то нужно здесь записать false

- msg: сообщение с пояснениями. Сообщение должно быть одним из следующих: Бронирование создано, Комната занята, Комната не найдена. Сообщение выбирается на основе того, что вернет функция register_booking

- booking: это бронирование в виде json-строки. Должны содержаться поля: room_name, duration, start_date, end_date, start_time, end_time.

###**Пример использования**
```
result = create_booking(
    "Вагнер",
    datetime.datetime(2022, 9, 1, 14),
    datetime.datetime(2022, 9, 1, 15, 15)
)
print(result)
```

Функция возвращает:
```
{
  "created": false,
  "msg": "Комната занята",
  "booking": {
    "room_name": "Вагнер",
    "start_date": "2022-09-01",
    "start_time": "14:00",
    "end_date": "2022-09-01",
    "end_time": "15:15",
    "duration": 75
  }
}
```

###**Примечания**

Пример написания функции `create_booking`:
```
from api import register_booking

def create_booking(room_name, start, end):
    booking = Booking(........)
    try:
        result = register_booking(booking)
    except ....:
        ....

    return json.dumps(......)
```

In [ ]:
import json
import datetime
from api import register_booking

class Booking:
    def __init__(self, room_name, start, end):
        if end < start:
            raise ValueError("Время окончания бронирования не может быть раньше времени начала.")

        self.room_name = room_name
        self.start = start
        self.end = end

    @property
    def duration(self):
        return int((self.end - self.start).total_seconds() // 60)

    @property
    def start_date(self):
        return self.start.strftime("%Y-%m-%d")

    @property
    def end_date(self):
        return self.end.strftime("%Y-%m-%d")

    @property
    def start_time(self):
        return self.start.strftime("%H:%M")

    @property
    def end_time(self):
        return self.end.strftime("%H:%M")

def create_booking(room_name, start, end) -> str:
    print("Начинаем создание бронирования")
    booking = None
    try:
        booking = Booking(room_name, start, end)
        result = register_booking(booking)

        if result is True:
            msg = "Бронирование создано"
            created = True
        elif result is False:
            msg = "Комната занята"
            created = False
    except KeyError:
        msg = "Комната не найдена"
        created = False
    except ValueError as ve:
        msg = str(ve)
        created = False
    finally:
        print("Заканчиваем создание бронирования")

    booking_info = {
        "room_name": booking.room_name if booking else room_name,
        "duration": booking.duration if booking else 0,
        "start_date": booking.start_date if booking else "",
        "end_date": booking.end_date if booking else "",
        "start_time": booking.start_time if booking else "",
        "end_time": booking.end_time if booking else ""
    }

    response = {
        "created": created,
        "msg": msg,
        "booking": booking_info
    }

    return json.dumps(response)

